In [1]:
from funlib.persistence import open_ds
import numpy as np
from scipy.spatial import KDTree
from skimage.segmentation import find_boundaries
import pandas as pd

plasmodesmata_df = pd.read_csv(
    "/nrs/cellmap/ackermand/cellmap/analysisResults/leaf-gall/jrc_22ak351-leaf-3m.n5/fragments_relabeled.csv"
)
columns = open_ds(
    "/nrs/cellmap/ackermand/cellmap/leaf-gall/jrc_22ak351-leaf-3m.n5",
    "plasmodesmata_column_cells",
)
data = columns.to_ndarray()
targets = open_ds(
    "/nrs/cellmap/ackermand/cellmap/leaf-gall/jrc_22ak351-leaf-3m.n5",
    "plasmodesmata_column_target_cells",
)
targets_data = targets.to_ndarray()
targets_data[targets_data > 0] += data.max()
data += targets_data

# get surface voxels
from scipy.ndimage import binary_erosion

surface_voxels = find_boundaries(data, mode="inner")
cell_surface_voxels_coords = np.argwhere(surface_voxels) * columns.voxel_size
plasmodesmata_coms = np.array(
    [
        plasmodesmata_df["COM Z (nm)"] - 8 + 128,
        plasmodesmata_df["COM Y (nm)"] - 8 + 128,
        plasmodesmata_df["COM X (nm)"] - 8 + 128,
    ]
).T

# Create KD-trees for efficient distance computation
tree1 = KDTree(cell_surface_voxels_coords)
tree2 = KDTree(plasmodesmata_coms)

# Find all pairs of points from both organelles within the threshold distance
contact_voxels_list_of_lists = tree1.query_ball_tree(tree2, r=2000)

In [2]:
from tqdm import tqdm

plasmodesmata_ids = plasmodesmata_df["Object ID"].to_numpy()
plasmodesmata_to_cells_dict = dict(
    zip(plasmodesmata_ids, [{} for _ in range(len(plasmodesmata_ids))])
)

contact_voxels_pairs = [
    [i, j] for i, sublist in enumerate(contact_voxels_list_of_lists) for j in sublist
]

for contact_voxels_pair in tqdm(contact_voxels_pairs):
    cell_surface_voxel_index = contact_voxels_pair[0]
    cell_surface_voxel_coords = cell_surface_voxels_coords[cell_surface_voxel_index]
    # cell_surface_voxel_inds = tuple(cell_surface_voxel_coords // columns.voxel_size)
    plasmodesmata_index = contact_voxels_pair[1]
    dist = np.linalg.norm(
        plasmodesmata_coms[plasmodesmata_index] - cell_surface_voxel_coords
    )
    cell_id = data[tuple(cell_surface_voxel_coords // columns.voxel_size)]
    plasmodesmata_id = plasmodesmata_ids[plasmodesmata_index]
    cell_to_dist_dict = plasmodesmata_to_cells_dict.get(plasmodesmata_id)
    cell_to_dist_dict[cell_id] = min(cell_to_dist_dict.get(cell_id, np.inf), dist)
    # print(
    #     plasmodesmata_df.loc[plasmodesmata_index][
    #         ["COM X (nm)", "COM Y (nm)", "COM Z (nm)"]
    #     ].to_numpy(),
    #     plasmodesmata_df.loc[plasmodesmata_index][["Object ID"]].to_numpy(),
    #     data[cell_surface_voxel_coords],
    # )

100%|██████████| 23390701/23390701 [03:58<00:00, 97925.65it/s] 


In [104]:
plasmodesmata_coms[plasmodesmata_df["Object ID"] == 8589934594]

array([[22835.96577755, 21803.29340715, 20446.22848515]])

In [103]:
plasmodesmata_coms[8589934594]

IndexError: index 8589934594 is out of bounds for axis 0 with size 391068

In [97]:
# create array of empty dicts


temp = [{} for _ in range(10)]
temp[0]["a"] = 4
temp

[{'a': 4}, {}, {}, {}, {}, {}, {}, {}, {}, {}]

In [75]:
cell_surface_voxel_coords

array([ 74240,  38912, 206848])

In [29]:
data[tuple(cell_surface_voxel // ds.voxel_size)]

1

In [26]:
tuple(cell_surface_voxel // ds.voxel_size)

(93, 32, 55)